# GeekBrains University
## Methods of data collection

## Lesson 3
### MongoDB and SQLite Database Management Systems in Python

### The Task 
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать
функцию, записывающую собранные вакансии в созданную БД.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной
платой больше введённой суммы.
3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с
сайта.

In [93]:
from bs4 import BeautifulSoup as bs
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient

#### Fetch the number of pages (see the homework Lesson 2)

In [94]:
def pages_counter_hh(main_link,  occupation= 'санитар', headers= {}):
    
    #request for the first page
    response = requests.get(f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}', headers=headers).text
    soup = bs(response,'lxml')
    
    # Look for the number of pages on the footer
    pages = soup.find_all('a', {'class' : "bloko-button HH-Pager-Control"})
    pages = int(pages[-1].text)
    
    return(pages)

In [95]:
def pages_counter_sj(main_link,  occupation= 'санитар', headers= {}):
    
    #request for the first page
    response = requests.get(f'{main_link}/vacancy/search/?keywords={occupation}', headers=headers).text
    soup = bs(response,'lxml')
    
    # Look for the number of pages on the footer
    pages = soup.find('div', {'class' : "L1p51"}).find_all('span', {'class' : "_3IDf-"})
    pages = int(pages[-2].text)
    
    return(pages)

#### Parcers for every vacancy (see the homework Lesson 2)

In [96]:
def parcer_hh(vacancy, main_link):
    v_data = {}    
    
    title = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not title:
        v_data['title'] = np.nan
    else:
        v_data['title'] = title.text
    
    link = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not link:
        v_data['link'] = np.nan
    else:
        v_data['link'] = link['href']
        
    employer = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-employer'})
    if not employer:
        v_data['employer'] = np.nan
    else:
        v_data['employer'] = employer.text
    
    compensation = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
    if not compensation:
        v_data['compensation'] = np.nan
    else:
        v_data['compensation'] = compensation.text
    
    v_data['source'] = main_link
    
    return(v_data)

In [97]:
def parcer_sj(vacancy, main_link):
    
    v_data = {}
    
    title = vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    if not title:
        v_data['title'] = np.nan
    else:
        title = title.find('a')
        if not title:
            v_data['title'] = np.nan
        else:
            v_data['title'] = title.text
    
    link = vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    if not link:
        v_data['link'] = np.nan
    else:
        link = link.find('a')
        if not link:
            v_data['link'] = np.nan
        else:
            v_data['link'] = main_link + link['href']
    
    employer = vacancy.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'})
    if not employer:
        v_data['employer'] = np.nan
    else:
        employer = employer.find('a')
        if not employer:
            v_data['employer'] = np.nan
        else:
            v_data['employer'] = employer.text
    
    compensation = vacancy.find('span', {'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    if not compensation:
        v_data['compensation'] = np.nan
    else:
        v_data['compensation'] = compensation.text
        
    v_data['source'] = main_link
    
    return(v_data)

#### Parcer for compensation string (min, max, currency) (see the homework Lesson 2)

In [98]:
def compensation_parser(v_data):
    v_data['compensation_min'] = np.nan
    v_data['compensation_max'] = np.nan
    v_data['compensation_currency'] = np.nan
    
    if v_data['compensation'] == np.nan:
        return(v_data)
    
    compensation_tt = str(v_data['compensation'])
    compensation_tt = compensation_tt.replace(u'\xa0', ' ')
    
    numbers = '[\d+\s]*\d+' # pattern for number values
    
    fr = re.search(f'^от {numbers}', compensation_tt) # 'от 30 000'
    if fr:
        v_data['compensation_min'] = float(fr[0][3:].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
        
    fr = re.search(f'^{numbers}[\s]*[\-—]+', compensation_tt) # '30 000-'
    if fr:
        v_data['compensation_min'] = float(fr[0][:-1].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '-')
    
    fr = re.search(f'^до {numbers}', compensation_tt) # 'до 30 000'
    if fr:
        v_data['compensation_max'] = float(fr[0][3:].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
        
        
    fr = re.search(f'^[\-—]+[\s]*{numbers}', compensation_tt) # '-30 000'
    if fr:
        v_data['compensation_max'] = float(fr[0][1:].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
        
    fr = re.search(f'^[\s]*{numbers}', compensation_tt) # '30 000'
    if fr:
        v_data['compensation_max'] = float(fr[0].replace(' ', ''))
        v_data['compensation_min'] = float(fr[0].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
    
    v_data['compensation_currency'] = compensation_tt # the rest to the currency
    
    return(v_data)

---

#### Function for comparing the new vacancy data with data of vacancies, existing in the base

In [99]:
def vacancy_comparing(collection, v_data):
    
    # Return '0' for existing document and the existing document data
    if collection.count_documents(v_data):
        return(0, collection.find(v_data)[0]) 
    
    #Return '-1' for a changed document: the 'new' vacancy has the same link as the one in the base
    elif collection.count_documents({'link' : v_data['link']}):
        return(-1, collection.find({'link' : v_data['link']})[0])
    
    # Return '1' for the vacancy with a new link
    else:
        return(1, {})

#### Function insert completely new and update (if it's neccessary) partially new vacancies for given site (links)

In [100]:
def db_insert_update(collection, pages, link, vacancy_pattern, main_link, headers= {},
                     update= True, the_first_page_only= False):
    
    counter = np.zeros(2) # Let the function return the number of completely new and updated vacancies
    
    n_pages = (pages, 1)[the_first_page_only]
    
    for page in range(n_pages):
        
        # Get the soup part for every vacancy
        link = link + f'{page}'
        response = requests.get(link, headers=headers).text
        soup = bs(response,'lxml')
        vacancies = soup.find_all('div', vacancy_pattern)
        
        for vacancy in vacancies:
            # Parse the required field
            v_data = parcer_hh(vacancy, main_link)
            v_data = compensation_parser(v_data)

            comparing_result = vacancy_comparing(collection, v_data)
            
            if comparing_result[0] == 0: # No action for existing data
                continue
                
            elif comparing_result[0] == 1: # Insert completely new vacancy
                collection.insert_one(v_data)
                counter[0] += 1
                
            elif update: # Update changed vacancy if parametr 'update' is True 
                collection.update_one(comparing_result[1], {'$set' : v_data})
                counter[1] += 1

    return counter

#### Function aggregate the job of 'db_insert_update' function for both sites 

In [101]:
def db_insert_update_two_site(collection, occupation= 'санитар', headers= {}, update= True, the_first_page_only= False):
    
    counter = np.zeros(2) # Let the function return the number of completely new and updated vacancies
    
    # The HeadHunter part:
    main_link = 'https://hh.ru'
    pages = pages_counter_hh(main_link,  occupation, headers)
    link = f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}&page='
    vacancy_pattern = {'data-qa':'vacancy-serp__vacancy'}
           
    counter += db_insert_update(collection, pages, link, vacancy_pattern, main_link, headers, update, the_first_page_only)
    
    #---------------------------------------------------------------------------------------------------------
    # The SuperJob part:
    main_link = 'https://www.superjob.ru'
    pages = pages_counter_sj(main_link,  occupation, headers)
    link = f'{main_link}/vacancy/search/?keywords={occupation}&page='
    vacancy_pattern = {'class':'_3zucV f-test-vacancy-item _3j3cA RwN9e _3tNK- _1NStQ _1I1pc'}
    
    counter += db_insert_update(collection, pages, link, vacancy_pattern, main_link, headers, update, the_first_page_only)
    
    return counter

---

### The main solution

In [102]:
client = MongoClient( 'localhost' , 27017 )
db = client['vacancies']
collection = db.vacancies_3

In [103]:
headers = {'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
occupation = 'санитар'

#### Create the base partially for the first pages only (to check insert and update later)

In [104]:
collection.drop({})

In [105]:
update= True
the_first_page_only= True
counter = db_insert_update_two_site(collection, occupation, headers, update, the_first_page_only)
total_instances = collection.count_documents({})
print(f' Completely new instances {counter[0]} \n Updated instances {counter[1]} \n Total number of instances {total_instances}')

 Completely new instances 62.0 
 Updated instances 0.0 
 Total number of instances 62


#### Insert and update

In [106]:
update= True
the_first_page_only= False
counter = db_insert_update_two_site(collection, occupation, headers, update, the_first_page_only)
total_instances = collection.count_documents({})
print(f' Completely new instances {counter[0]} \n Updated instances {counter[1]} \n Total number of instances {total_instances}')

 Completely new instances 39.0 
 Updated instances 0.0 
 Total number of instances 101


#### Slicing

In [113]:
required_compensation = 30000 
result = collection.find({'compensation_min':{'$gt' : required_compensation}}).sort('compensation_min', -1)
df = pd.DataFrame.from_dict(result)

df

,_id,compensation,compensation_currency,compensation_max,compensation_min,employer,link,source,title
0,5e94f12c3525a516f2834c60,от 200 000 руб.,руб.,NaN,200000.0,ООО Центрпроект,https://hh.ru/vacancy/36394467?query=%D1%81%D0...,https://hh.ru,Главный инженер по строительству
1,5e94f0c53525a516f2834c2e,от 150 000 руб.,руб.,NaN,150000.0,ООО Стимул Групп,https://hh.ru/vacancy/36587033?query=%D1%81%D0...,https://hh.ru,Прораб
2,5e94f0c53525a516f2834c1b,от 100 000 руб.,руб.,NaN,100000.0,ООО Гарант-Мед,https://hh.ru/vacancy/36601291?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар
3,5e94f0c53525a516f2834c37,от 100 000 руб.,руб.,NaN,100000.0,ООО Астра,https://hh.ru/vacancy/36307101?query=%D1%81%D0...,https://hh.ru,Директор по строительству
4,5e94f12c3525a516f2834c66,от 100 000 руб.,руб.,NaN,100000.0,Design destiny,https://hh.ru/vacancy/36263138?query=%D1%81%D0...,https://hh.ru,Главный инженер проектов
5,5e94f0c53525a516f2834c49,от 80 000 руб.,руб.,NaN,80000.0,ООО Каравай СВ,https://hh.ru/vacancy/36531420?query=%D1%81%D0...,https://hh.ru,Технолог общественного питания в хлебопекарню
6,5e94f12c3525a516f2834c5f,от 80 000 руб.,руб.,NaN,80000.0,ООО НоаТек,https://hh.ru/vacancy/36260437?query=%D1%81%D0...,https://hh.ru,Прораб
7,5e94f12c3525a516f2834c63,от 80 000 руб.,руб.,NaN,80000.0,ООО Сантоп-Трейдинг,https://hh.ru/vacancy/36421619?query=%D1%81%D0...,https://hh.ru,Заместитель начальника склада
8,5e94f0c53525a516f2834c27,70 000-90 000 руб.,руб.,90000.0,70000.0,ООО Виллетта,https://hh.ru/vacancy/35066636?query=%D1%81%D0...,https://hh.ru,Санитарный врач
9,5e94f0c53525a516f2834c31,70 000-80 000 руб.,руб.,80000.0,70000.0,ООО Атриум,https://hh.ru/vacancy/36376180?query=%D1%81%D0...,https://hh.ru,Геодезист
